# Boilerplate

In [3]:
import pandas as pd 
import pandasql as ps
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import scipy.stats as stats
import statsmodels.stats.api as sms

from IPython.display import display
from IPython.display import Markdown as md
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

def f():
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    
def nf():
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 4)

import warnings
warnings.filterwarnings('ignore')

def dp(df, r = 5, c = None):
    from IPython.display import display
    with pd.option_context('display.max_rows', 4, 'display.max_columns', None):
        display(df)

def fg(w = 6, h = 4, dpi = 120):
    plt.rcParams['figure.figsize'] = (w,h)
    plt.rcParams['figure.dpi'] = dpi
fg()

%load_ext autoreload
%autoreload 2

# Create sample

In [4]:
import random 
import string 

In [200]:
# letters = string.ascii_uppercase
letters = string.ascii_uppercase[:5]

df = pd.DataFrame()
_len = 200
for i in range(5):
    df[f'Q{i}'] = [''.join(random.choice(letters) for _ in range(1)) for _ in range(_len)]

In [201]:
df

,Q0,Q1,Q2,Q3,Q4
0,C,A,C,B,A
1,E,C,E,D,C
2,C,E,A,C,C
3,D,C,C,A,A
4,A,E,D,A,D
...,...,...,...,...,...
195,B,E,D,D,A
196,D,B,C,E,C
197,D,D,E,D,B
198,E,C,E,E,A


# random sample selection

In [202]:
n_clu = 3
_cho = random.sample(df.index.to_list(),n_clu)

dfcho = df[df.index.isin(_cho)].reset_index(drop = True)

In [203]:
dforig = df.copy()

# Compute distances

In [204]:
# computacao ineficient O(n²) ou mais
for idx in df.index:
    for clu in dfcho.index:
        df.loc[idx,f'C{clu}'] = sum(dforig.iloc[idx,:] == dfcho.iloc[clu,:])

In [205]:
df

,Q0,Q1,Q2,Q3,Q4,C0,C1,C2
0,C,A,C,B,A,1.0,0.0,1.0
1,E,C,E,D,C,3.0,0.0,0.0
2,C,E,A,C,C,1.0,1.0,1.0
3,D,C,C,A,A,3.0,1.0,1.0
4,A,E,D,A,D,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...
195,B,E,D,D,A,1.0,0.0,1.0
196,D,B,C,E,C,3.0,1.0,2.0
197,D,D,E,D,B,2.0,2.0,0.0
198,E,C,E,E,A,1.0,0.0,1.0


# Minimum distances to clusters

In [206]:
dfclosest = df.select_dtypes(include = [float]).idxmin(axis = 1)

In [217]:
closest = 'closest'
df[closest] = dfclosest

In [208]:
df

,Q0,Q1,Q2,Q3,Q4,C0,C1,C2,closest
0,C,A,C,B,A,1.0,0.0,1.0,C1
1,E,C,E,D,C,3.0,0.0,0.0,C1
2,C,E,A,C,C,1.0,1.0,1.0,C0
3,D,C,C,A,A,3.0,1.0,1.0,C1
4,A,E,D,A,D,0.0,0.0,3.0,C0
...,...,...,...,...,...,...,...,...,...
195,B,E,D,D,A,1.0,0.0,1.0,C1
196,D,B,C,E,C,3.0,1.0,2.0,C1
197,D,D,E,D,B,2.0,2.0,0.0,C2
198,E,C,E,E,A,1.0,0.0,1.0,C1


In [209]:
clusters = df.select_dtypes(include = [float]).columns

In [210]:
for clu in clusters:
    df[df.closest == clu]

,Q0,Q1,Q2,Q3,Q4,C0,C1,C2,closest
2,C,E,A,C,C,1.0,1.0,1.0,C0
4,A,E,D,A,D,0.0,0.0,3.0,C0
5,E,E,A,B,D,0.0,0.0,2.0,C0
6,A,B,B,E,C,1.0,1.0,2.0,C0
9,A,A,A,C,E,0.0,2.0,1.0,C0
...,...,...,...,...,...,...,...,...,...
187,E,E,D,C,A,0.0,1.0,1.0,C0
190,C,B,D,B,E,0.0,1.0,0.0,C0
191,E,B,E,C,A,0.0,1.0,0.0,C0
193,A,D,E,D,A,1.0,1.0,1.0,C0


,Q0,Q1,Q2,Q3,Q4,C0,C1,C2,closest
0,C,A,C,B,A,1.0,0.0,1.0,C1
1,E,C,E,D,C,3.0,0.0,0.0,C1
3,D,C,C,A,A,3.0,1.0,1.0,C1
7,E,B,D,D,B,1.0,0.0,0.0,C1
8,A,A,A,B,C,1.0,0.0,1.0,C1
...,...,...,...,...,...,...,...,...,...
188,B,B,D,D,C,2.0,0.0,0.0,C1
189,A,E,C,E,D,1.0,0.0,5.0,C1
195,B,E,D,D,A,1.0,0.0,1.0,C1
196,D,B,C,E,C,3.0,1.0,2.0,C1


,Q0,Q1,Q2,Q3,Q4,C0,C1,C2,closest
10,C,B,D,D,E,1.0,1.0,0.0,C2
22,D,B,A,B,E,1.0,2.0,0.0,C2
25,E,D,D,D,B,1.0,1.0,0.0,C2
35,D,B,A,C,A,1.0,2.0,0.0,C2
40,C,C,D,C,E,1.0,2.0,0.0,C2
62,D,B,C,C,C,3.0,2.0,1.0,C2
74,B,B,A,C,C,1.0,1.0,0.0,C2
77,B,A,B,D,B,1.0,1.0,0.0,C2
80,B,C,E,C,A,1.0,1.0,0.0,C2
90,D,B,B,A,C,2.0,2.0,0.0,C2


# Return mode

In [211]:
df

,Q0,Q1,Q2,Q3,Q4,C0,C1,C2,closest
0,C,A,C,B,A,1.0,0.0,1.0,C1
1,E,C,E,D,C,3.0,0.0,0.0,C1
2,C,E,A,C,C,1.0,1.0,1.0,C0
3,D,C,C,A,A,3.0,1.0,1.0,C1
4,A,E,D,A,D,0.0,0.0,3.0,C0
...,...,...,...,...,...,...,...,...,...
195,B,E,D,D,A,1.0,0.0,1.0,C1
196,D,B,C,E,C,3.0,1.0,2.0,C1
197,D,D,E,D,B,2.0,2.0,0.0,C2
198,E,C,E,E,A,1.0,0.0,1.0,C1


In [212]:
aux = pd.DataFrame({'a':[1,2,3,2,1,2,3], 'b':[1,4,5,6,4,5,6]})

In [213]:
aux.mode(axis = 0)

,a,b
0,2.0,4
1,NaN,5
2,NaN,6


In [214]:
df['Q0'].mode()

0    A
dtype: object

In [215]:
df.mode(axis = 0)

,Q0,Q1,Q2,Q3,Q4,C0,C1,C2,closest
0,A,B,C,C,E,1.0,0.0,1.0,C0
1,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
dforig[closest] = df[closest]

In [221]:
dfmode = pd.DataFrame()
for clu in clusters:
    dfmode = dfmode.append(dforig[dforig.closest == clu].mode(axis = 0).iloc[0,:])
dfmode = dfmode.drop(closest, axis = 1)

In [222]:
dfmode

,Q0,Q1,Q2,Q3,Q4
0,A,B,B,C,E
0,E,C,C,D,C
0,D,C,B,C,E
